In [1]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from youtube_transcript_api import YouTubeTranscriptApi
from typing import Optional
import re

will continue: https://medium.com/@sahin.samia/how-to-build-an-ai-powered-youtube-video-summariser-app-with-llama-3-2-and-langchain-28c2b6df76da

# Extract transcript from YouTube

In [2]:
from youtube_transcript_api import YouTubeTranscriptApi


def get_youtube_transcript(video_id):
    """
    Retrieves the transcript of a YouTube video given its video ID.

    Args:
    video_id: The ID of the YouTube video.

    Returns:
    A list of dictionaries, where each dictionary represents a transcript segment,
    or None if an error occurs or the transcript is not available. Each dictionary
    contains the keys 'text', 'start', and 'duration'.
    """
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


# Example usage: Replace 'YOUR_VIDEO_ID' with the actual video ID.
video_id = '5K1-lIv1VAU'
transcript = get_youtube_transcript(video_id)

if transcript:
    for segment in transcript:
        print(f"[{segment['start']:.2f} - {segment['start'] + segment['duration']:.2f}] {segment['text']}")
else:
    print("Could not retrieve transcript.")

[0.60 - 3.27] (gentle music)
[13.17 - 16.03] ♪ Little girl at a lonely well ♪
[16.07 - 19.10] ♪ With the same little tale to
tell ♪
[19.13 - 22.13] ♪ Feeling trapped by the
walls that hold her ♪
[22.17 - 25.17] ♪ Feeling stuck in the
story they've told her ♪
[25.20 - 28.20] ♪ Another day where she fades
away ♪
[28.23 - 30.53] ♪ Never daring to disobey ♪
[30.57 - 33.53] ♪ So she's dreaming all alone ♪
[33.57 - 37.67] ♪ Sharing secrets with the stone
♪
[37.70 - 40.53] ♪ My father told me long ago ♪
[40.57 - 44.07] ♪ I braved a bitter storm of
snow ♪
[44.10 - 49.10] ♪ Is that a girl I'll ever know
again ♪
[50.43 - 52.23] ♪ I'm waiting on a wish ♪
[52.27 - 55.07] ♪ Beneath a thousand treetops ♪
[55.10 - 57.50] ♪ And as the silver sky stops ♪
[57.53 - 62.13] ♪ I send a whisper to the water
♪
[62.17 - 63.53] ♪ Waiting on a wish ♪
[63.57 - 66.37] ♪ Holding out for someday ♪
[66.40 - 69.17] ♪ Hoping somehow, some way ♪
[69.20 - 73.30] ♪ I'll become my father's
daughter ♪
[73.33 - 76.10] ♪ I cl

In [4]:
all_texts = ""
for i in transcript:
    all_texts = all_texts + i['text'] + " "
all_texts

'(gentle music) ♪ Little girl at a lonely well ♪ ♪ With the same little tale to\ntell ♪ ♪ Feeling trapped by the\nwalls that hold her ♪ ♪ Feeling stuck in the\nstory they\'ve told her ♪ ♪ Another day where she fades\naway ♪ ♪ Never daring to disobey ♪ ♪ So she\'s dreaming all alone ♪ ♪ Sharing secrets with the stone\n♪ ♪ My father told me long ago ♪ ♪ I braved a bitter storm of\nsnow ♪ ♪ Is that a girl I\'ll ever know\nagain ♪ ♪ I\'m waiting on a wish ♪ ♪ Beneath a thousand treetops ♪ ♪ And as the silver sky stops ♪ ♪ I send a whisper to the water\n♪ ♪ Waiting on a wish ♪ ♪ Holding out for someday ♪ ♪ Hoping somehow, some way ♪ ♪ I\'ll become my father\'s\ndaughter ♪ ♪ I close my eyes and see ♪ ♪ The girl I\'m meant to be ♪ ♪ Is she a part of me ♪ ♪ I\'ve yet to find ♪ ♪ Wondering, "will she appear" ♪ ♪ Or will I be forever here ♪ ♪ Waiting on a wish ♪ (gentle music continues) ♪ Little girl by a wild wood ♪ ♪ How she tries to be someone\ngood ♪ ♪ But in the shadow the\nkingdom\'s caugh

In [ ]:
#Example of how to save it to a text file.
def save_transcript_to_file(transcript, filename="transcript.txt"):
    if transcript:
        with open(filename, "w", encoding="utf-8") as f:
            for segment in transcript:
                f.write(f"[{segment['start']:.2f} - {segment['start'] + segment['duration']:.2f}] {segment['text']}\n")
        print(f"Transcript saved to {filename}")
    else:
        print("No transcript to save.")

save_transcript_to_file(transcript)

# Setup Langchain & vertex AI

In [6]:
%pwd

'/Users/picetrp/Documents/Learn/My_Self_Learn/llm_hub/youtube_trend_analysis'

In [8]:
# setup langchain & langsmith
# setup langsmith

import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv
    print("load environment variables")
    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
    
# vertex AI
if "GOOGLE_APPLICATION_CREDENTIALS" not in os.environ:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = getpass.getpass(
        prompt='Enter your GOOGLE_APPLICATION_CREDENTIALS (.json path): '
    )

load environment variables


In [9]:
# Ensure your VertexAI credentials are configured

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai")

# Create Summarizer Class

In [ ]:
class YouTubeSummarizer:
    def __init__(self):
        """
        Initialize the YouTube Summarizer
        """
        
        self.llm = ChatOllama(temperature=0, model="llama3.2")
        
        # Initialize text splitter for long transcripts
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=10000,
            chunk_overlap=1000,
            separators=["\n\n", "\n", " ", ""]
        )
        
        # Custom prompts for the summary chain
        self.map_prompt_template = """
        Summarize the following part of a YouTube video transcript:
        "{text}"
        
        KEY POINTS AND TAKEAWAYS:
        """
        
        self.combine_prompt_template = """
        Create a detailed summary of the YouTube video based on these transcript summaries:
        "{text}"
        
        Please structure the summary as follows:
        1. Main Topic/Theme
        2. Key Points
        3. Important Details
        4. Conclusions/Takeaways
        
        DETAILED SUMMARY:
        """
        
        # Create the summary chain
        self.map_prompt = PromptTemplate(
            template=self.map_prompt_template,
            input_variables=["text"]
        )
        
        self.combine_prompt = PromptTemplate(
            template=self.combine_prompt_template,
            input_variables=["text"]
        )
        
        self.chain = load_summarize_chain(
            llm=self.llm,
            chain_type="map_reduce",
            map_prompt=self.map_prompt,
            combine_prompt=self.combine_prompt,
            verbose=False
        )